![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database
- Schema
- Queries
- Tables
- Views
- Primary Keys
- Foreign Keys
- Indexes
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

In [18]:
cur2 = con.cursor()
cur2.description

In [4]:
type(cursor)

sqlite3.Cursor

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [7]:
cursor.description

(('name', None, None, None, None, None, None),)

In [13]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [9]:
cursor.description

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

In [10]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [15]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [16]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [17]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [19]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [20]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [26]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [31]:
# how many rows in airlines?
cursor.execute("SELECT COUNT(*) airline_count FROM airlines a;")
info = cursor.fetchall()
print(*info, sep = "\n")

(6048,)


In [32]:
cursor.description

(('airline_count', None, None, None, None, None, None),)

In [33]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [34]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

In [35]:
cursor.execute("PRAGMA table_info(airports)").fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

In [36]:
cursor.execute("select * from airports limit 5").fetchall()

[(0,
  '1',
  'Goroka',
  'Goroka',
  'Papua New Guinea',
  'GKA',
  'AYGA',
  '-6.081689',
  '145.391881',
  '5282',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (1,
  '2',
  'Madang',
  'Madang',
  'Papua New Guinea',
  'MAG',
  'AYMD',
  '-5.207083',
  '145.7887',
  '20',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby')]

In [40]:
cursor.execute("""
SELECT DISTINCT country
FROM airports
ORDER BY 1 DESC
""").fetchmany(10)

[('Zimbabwe',),
 ('Zambia',),
 ('Yemen',),
 ('Western Sahara',),
 ('West Bank',),
 ('Wallis and Futuna',),
 ('Wake Island',),
 ('Virgin Islands',),
 ('Vietnam',),
 ('Venezuela',)]

In [41]:
# LIST OF COUNTRIES SORTED BY # OF AIRPORTS IN THAT COUNTRY
cursor.execute("""
SELECT country, count(*) AS airport_count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
""").fetchmany(10)

[('United States', 1697),
 ('Canada', 435),
 ('Germany', 321),
 ('Australia', 263),
 ('Russia', 249),
 ('France', 233),
 ('China', 219),
 ('Brazil', 213),
 ('United Kingdom', 210),
 ('India', 140)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [50]:
cursor.execute("""
SELECT
  r.*,
  a_source.latitude source_lat,
  a_source.longitude source_lon,
  a_dest.latitude dest_lat,
  a_dest.longitude dest_lon

FROM routes r
LEFT JOIN airports a_source ON r.source_id = a_source.id
LEFT JOIN airports a_dest ON r.dest_id = a_dest.id
limit 1
""")
print(cursor.fetchall(), sep='\n')

[(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2', '43.449928', '39.956589', '55.606186', '49.278728')]


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [54]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC

LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)


In [55]:
# Which countries have the highest amount of inactive airlines?
cursor.execute("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY 2 DESC

LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)


In [57]:
# What about airports by timezones?
cursor.execute("""
SELECT timezone, count(*) as tz_count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)


In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

In [66]:
f = open('flights.db', 'r')

In [67]:
f.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [58]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [64]:
max_data = shelter_data.query('Name == "Spot" and Color == "Tricolor"')
max_data.head()
#vs
shelter_data[(shelter_data['Name']=="Spot") & (shelter_data['Color']=='Tricolor')]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
6188,A783690,Spot,06/10/2019 02:26:00 PM,06/10/2019 02:26:00 PM,11/05/2000,Return to Owner,NaN,Dog,Intact Male,18 years,Jack Russell Terrier Mix,Tricolor
16262,A783690,Spot,11/05/2018 05:27:00 PM,11/05/2018 05:27:00 PM,11/05/2000,Return to Owner,NaN,Dog,Intact Male,18 years,Jack Russell Terrier Mix,Tricolor
104218,A668894,Spot,12/11/2013 03:31:00 PM,12/11/2013 03:31:00 PM,12/11/2001,Return to Owner,NaN,Dog,Neutered Male,12 years,Rat Terrier Mix,Tricolor


In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [65]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [68]:
# Which countries have the highest amount of active airports?
pd.read_sql_query("""
SELECT country, count(*) as airline_count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY 2 DESC

LIMIT 5
""", pd_con)

,country,airline_count
0,United States,939
1,Mexico,427
2,United Kingdom,367
3,Canada,284
4,Russia,158


In [69]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [70]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [71]:
# How many tables are in the database?
cur.execute("SELECT count(*) as table_count FROM sqlite_master WHERE type = 'table';").fetchone()

(11,)

In [72]:
# What's the primary key of each table?

table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in [x[0] for x in table_names]:
    results = cur.execute(f"PRAGMA table_info({table_name});").fetchall()

    # column name is 2nd column,
    # primary key is the 4th column, according to https://www.sqlite.org/pragma.html#pragma_table_info
    pk_column_name = [x[1] for x in results if x[3] == 1]
    print(f"{table_name}.{pk_column_name[0]}")

Album.AlbumId
Artist.ArtistId
Customer.CustomerId
Employee.EmployeeId
Genre.GenreId
Invoice.InvoiceId
InvoiceLine.InvoiceLineId
MediaType.MediaTypeId
Playlist.PlaylistId
PlaylistTrack.PlaylistId
Track.TrackId


In [83]:
# What foreign keys join the tables together?

table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in [x[0] for x in table_names]:
    results = cur.execute(f"PRAGMA foreign_key_list({table_name});").fetchall()
    for result in results:
        print(f"{table_name}.{result[3]} => {result[2]}.{result[4]}")

Album.ArtistId => Artist.ArtistId
Customer.SupportRepId => Employee.EmployeeId
Employee.ReportsTo => Employee.EmployeeId
Invoice.CustomerId => Customer.CustomerId
InvoiceLine.TrackId => Track.TrackId
InvoiceLine.InvoiceId => Invoice.InvoiceId
PlaylistTrack.TrackId => Track.TrackId
PlaylistTrack.PlaylistId => Playlist.PlaylistId
Track.MediaTypeId => MediaType.MediaTypeId
Track.GenreId => Genre.GenreId
Track.AlbumId => Album.AlbumId


In [85]:
# What are the max and min dates in the Invoice table?
pd.read_sql_query("""
SELECT
    max(InvoiceDate) AS max_date,
    min(InvoiceDate) AS min_date
FROM Invoice
""", conn)

,max_date,min_date
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [ ]:
# What tables would you need to answer "what is your most popular track?"

# Track (for track info)
# InvoiceLine (for track puchase quantity)

In [ ]:
# What values from each table?

# Track.TrackId, Track.Name, Track.AlbumId
# InvoiceLine.TrackId, InvoiceLine.Quantity
# Album.AlbumId, Album.ArtistId
# Artist.ArtistId, Artist.Name

In [86]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

pd.read_sql_query("""
SELECT
    t.name AS track_name,
    sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""", conn)

,track_name,purchase_count
0,The Trooper,5
1,Eruption,4
2,Hallowed Be Thy Name,4
3,Sure Know Something,4
4,The Number Of The Beast,4
5,Untitled,4
6,2 Minutes To Midnight,3
7,Blood Brothers,3
8,Brasil,3
9,Can I Play With Madness,3


In [87]:
# Advanced: get the artist who sang the song!
pd.read_sql_query("""
SELECT
    t.name AS track_name,
    a.name AS artist_name,
    sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album USING (AlbumId)
JOIN Artist a USING (ArtistId)
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
""", conn)

,track_name,artist_name,purchase_count
0,The Trooper,Iron Maiden,5
1,Eruption,Van Halen,4
2,Hallowed Be Thy Name,Iron Maiden,4
3,Sure Know Something,Kiss,4
4,The Number Of The Beast,Iron Maiden,4
5,2 Minutes To Midnight,Iron Maiden,3
6,Blood Brothers,Iron Maiden,3
7,Can I Play With Madness,Iron Maiden,3
8,Dazed and Confused,Led Zeppelin,3
9,Flying High Again,Ozzy Osbourne,3


## Reflection

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1dZg9MqBmLqJHWupgSBfvhicrVT1vyTejtaz3sXYsgGk/edit)!